In [79]:
import ssl
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import time
import re
import statistics
import matplotlib.pyplot as py
import pandas as pd 
import random

In [89]:
def get_links(pages = 1):
    #gets the links from trulia properties page 
    # if length of list is zero, captcha blocked request so use a new VPN location 
    #pages: number of pages of trula to scrape
    #returns: list of links
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    i = 0
    all_property_links = []
    while i < pages:
        page_property_links = []
        url = 'https://www.trulia.com/NY/New_York/' + "%s_p/"%str(i)
        req = Request(url, headers={"User-Agent": 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage , 'html.parser')
        for link in soup.findAll('a'):
            l = str(link.get('href'))
            if sum(map(str.isdigit, l[-17:])) == 15: page_property_links.append("https://www.trulia.com" + l)
        page_property_links = page_property_links[::2]
        all_property_links.extend(page_property_links)
        print("finishing loop" + " " + str(i+1))
        i += 1
        print(len(all_property_links))
        time.sleep(random.randint(5,10))
        
    return(all_property_links)

In [90]:
links = get_links(pages = 10)
feature_dict_list = {}
all_features = []
all_links = []

finishing loop 1
54
finishing loop 2
108
finishing loop 3
144
finishing loop 4
183
finishing loop 5
221
finishing loop 6
261
finishing loop 7
298
finishing loop 8
335
finishing loop 9
369
finishing loop 10
407


In [58]:
import random
for index,url in enumerate(links): #iterates through all URLs from the get_links func
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()  
    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')
    feature_dict = {}

    for sou in soup.findAll('span', attrs={"class":"Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq"}):
        clean_data = str(sou).split("gmLKqq\">")
        clean_data = clean_data[1].split("</span>")[0]
        dict_info = clean_data.split(':')
        if len(dict_info)>1:
            feature_dict[dict_info[0]] = dict_info[1]

        for bed_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-bedrooms"}):
            number_beds = re.findall(r"[0-9]+\s[bB]eds*", str(bed_html))
            if number_beds:
                feature_dict["beds"] = int(number_beds[0].split(" ")[0])

        for bath_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-bathrooms"}):
            number_baths = re.findall(r"[0-9]+\s[bB]aths*", str(bath_html))
            if number_baths:
                feature_dict["baths"] = int(number_baths[0].split(" ")[0])

        for floor_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-floorspace"}):
            floor_footage = re.findall(r"[0-9]+,[0-9]+\ssqft*", str(floor_html))
            if floor_footage:
                feature_dict["floor_space"] = int(floor_footage[0].split(" ")[0].replace(",", ""))
    
        

    all_features.append(feature_dict);all_links.append(url)
    print(index/len(url))
    time.sleep(random.randint(5,10))

sleeping
sleeping
sleeping
sleeping
sleeping
sleeping
sleeping
sleeping
sleeping


KeyboardInterrupt: 

In [57]:
df = pd.DataFrame.from_dict(all_features)
pd.set_option('display.max_columns', None)
df

,Days on Market,Year Built,Property Type,Price Per Sqft,"<a href=""https",Units in Building,HOA Fee,Basement,Number of Rooms,Number of Bedrooms,Number of Bathrooms,Number of Bathrooms (full),Living Area,Foundation Area,Heating,Air Conditioning,Heating Fuel,Electric,Flooring,Stories,Floors,Roof,Foundation,Parking,Property Subtype,Architecture,Construction Materials,List Price,MLS Status,City,Community,MLS/Source ID,Building Area,Lot Area,Special Conditions,Types of Rooms,Appliances,Levels,Fencing,Number of Garage Spaces,Parking Spaces,Sewer,Year Renovated,Condition,Parcel Number,Listing Agreement Type,"<a href=""http",Water View,Number of Bathrooms (half),Common Walls,Patio / Porch,Exterior,Cooling System,Utilities,Window,Responsible for Road Maintenance,Elementary School District,Jr High / Middle School,Jr High / Middle School District,High School,High School District,Community Features,Attic,beds,baths,floor_space,Elementary School,Year Updated,Ownership,Laundry,Included in Sale,HOA Fee Includes,HOA Amenities,Accessibility,Structure Type
0,&lt;1 Day on Trulia,1964,Coop,$432,"//youtube.com/watch?v=sTAbBD2DiLI"" rel=""nofoll...",572,$814/Monthly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,&lt;1 Day on Trulia,1926,Coop,$974,NaN,186,"$2,522/Monthly",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,&lt;1 Day on Trulia,1950,Residential,$718,NaN,NaN,NaN,"Finished,Full",9,3,2,2,1224 Square Feet,612,"Natural Gas,Hot Water",Central Air,Natural Gas,110 V,Hardwood,2,Hardwood,Shingle,Other,"Shared Drive,Built-In Garage",Single Family Residence,Other,Brick,"$878,900",Active,Glendale,Queens,469335,1224 Square Feet,2300 sqft,Arm's Length,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,&lt;1 Day on Trulia,1980,Residential,$386,NaN,NaN,NaN,Full,5,3,1,1,2352 Square Feet,NaN,"Forced Air,Natural Gas",Central Air,Forced Air,220 Volts,NaN,2,NaN,NaN,NaN,"Built-in,Garage,Carport",Two Family - Semi-Attached,Colonial,NaN,"$908,838",Coming Soon,Staten Island,NaN,1158953,2352 Square Feet,3488 sqft,NaN,"Master Bedroom, Dining Room, Kitchen, Living ...","Refrigerator, Microwave",2.0,Fenced,1,1,Public Sewer,2019,"Existing, Excellent",4575-157,Exclusive Right To Sell,"//www.facebook/com/MickeyBaigs"" rel=""nofollow ...",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,&lt;1 Day on Trulia,1950,Residential,$445,NaN,NaN,NaN,Full,5,4,4,3,2359 Square Feet,NaN,"Hot Water,Natural Gas",Central Air,Hot Water,220 Volts,NaN,2,NaN,NaN,NaN,"Detached,Garage,Carport,On Street",Single Family - Detached,Colonial,NaN,"$1,049,000",Active,Staten Island,NaN,1158951,2359 Square Feet,5300 sqft,NaN,"Master Bedroom, Dining Room, Kitchen, Living ...","Dishwasher, Dryer, Microwave, Refrigerator, W...",2.0,Fenced,2,2,Public Sewer,2020,"Existing, Excellent",4604-0039,Exclusive Right To Sell,NaN,None,1,End Unit,Deck,Balcony,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,&lt;1 Day on Trulia,1969,Coop,NaN,NaN,187,"$1,410/Monthly",NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22156021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,&lt;1 Day on Trulia,1956,Coop,$850,NaN,287,"$1,967/Monthly",NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,"Tile, Carpe

In [ ]:
for house in all_features:
    for f in house:
        dict_key,dict_value = f.split(':')

In [62]:
# For ignoring SSL certificate errors  (reused code)

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [63]:
#url = input("Enter URL: ")
url = 'https://www.trulia.com/p/ny/staten-island/365-victory-blvd-staten-island-ny-10301--2008726710'

In [28]:
# Use Mozilla access agent

req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

In [29]:
# Creating a BeautifulSoup object of the html page for easy extraction of data.

soup = BeautifulSoup(webpage, 'html.parser')
html = soup.prettify('utf-8')
property_info = {}

In [30]:
# This code block will get you a one liner description of the listed property

for meta in soup.findAll('meta', attrs={'name': 'description'}):
    try:
        property_info['description'] = meta['content']
        break
    except: 
        pass

property_info['description']

KeyError: 'description'

In [31]:
#Store url 

property_info['link'] = url

property_info['link']

'https://www.trulia.com/p/ny/new-york/309-e-houston-st-1a-new-york-ny-10002--2478352572'

In [33]:
#Get beds, baths, floorspace

for bed_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-bedrooms"}):
    number_beds = re.findall(r"[0-9]+\s[bB]eds*", str(bed_html))
    if number_beds:
        property_info["beds"] = int(number_beds[0].split(" ")[0])

for bath_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-bathrooms"}):
    number_baths = re.findall(r"[0-9]+\s[bB]aths*", str(bath_html))
    if number_baths:
        property_info["baths"] = int(number_baths[0].split(" ")[0])

for floor_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-floorspace"}):
    floor_footage = re.findall(r"[0-9]+,[0-9]+\ssqft*", str(floor_html))
    property_info["floor_space"] = {}
    if floor_footage:
        property_info["floor_space"]["value"] = int(floor_footage[0].split(" ")[0].replace(",", ""))
        property_info["floor_space"]["unit"] = floor_footage[0].split(" ")[1]

property_info


{'link': 'https://www.trulia.com/p/ny/new-york/309-e-houston-st-1a-new-york-ny-10002--2478352572'}

In [69]:
#Extracts features for given house
for index, sou in enumerate(soup.findAll('span', attrs={"class":"Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq"})):
    clean_data = str(sou).split("gmLKqq\">")
    clean_data = clean_data[1].split("</span>")[0]
clean_data

'Lot Area: 3001 sqft'